## 2019 Data Science Bowl

이 노트북은 다음 노트북 내용을 분석 및 정리한 것입니다 :
    https://www.kaggle.com/mhviraf/a-new-baseline-for-dsb-2019-catboost-model
   

#### 요약 :
1. 이벤트 목록을 각각의 session마다 accuracy_group을 구하는 함수를 만든다. ------ A(x)
2. train 데이터를 위 함수(A)를 사용해서 변환한다. ------ a = A(train)
3. a의 데이터에서 accuracy_group을 y, 나머지 항목을 X로 해서 model을 훈련한다.
4. test데이터로 b = A(test)를 구해서 위 모델로 predict한다.


#### Submission :
test_set의 각각의 installation_id에 존재하는 마지막 assessment마다 accuracy_group을 예측해야 한다.
파일은 installation_id, accuracy_group의 두 column으로 이루어져 있어야 한다.

## EDA

In [ ]:
import numpy as np
import pandas as pd
import datetime
from catboost import CatBoostClassifier
from time import time
from tqdm import tqdm_notebook as tqdm

import subprocess
from ast import literal_eval

submission 평가는 QWK(Quadratic Weighted Kappa)로 이루어진다.

**Cohen's Kappa**

    두 연구자간 동일한 결과를 내놓는지를 수치화하는 방법이다.
    더 자세히 설명하면, 두 연구자 간 일치한 결과 중에서 우연히 일치할 가능성를 제외하고, 실제로 평가가 일치한 결과가 어느 정도인지 보여주는 지표이다.
    nominal(category간 거리가 같은)한 범주에 사용된다.

**Cohen's weighted Kappa**

    Cohen's Kappa와는 다르게, ordinal(순서가 있는, 예를 들어 관절염의 5 단계(1:없음, 2:경증 ... 5:심각) 등을 표현 시) 변수를 대상으로 할 경우에는 Cohen's weighted Kappa를 사용한다.<br>
    순서(또는 단계)가 있는 변수를 판단하므로 범주(카테고리)간 거리는 서로 다르고, 두 연구자간 결과가 다를 경우에도 다름의 크기에 가중치 차이가 있을 것이다.<br>
    이런 식으로 각각 다른 비중(weight)를 두어 불일치 정도를 평가하는 것이다.

    각 범주간 차이에 비중을 부여하는 방법으로는 값의 차이를 그대로 사용하는 linear 방법과, 제곱해서 사용하는 quadratic 방법이 있다.
    (1과 3의 차이 : linear = 2, quadratic = 4)

In [ ]:
from sklearn.metrics import confusion_matrix
def qwk(act,pred,n=4,hist_range=(0,3)):
    '''
    cohen's kappa는 두 데이터의 값 중 우연에 의해 일치하는 부분을 제외하고 
    실제 평가에 의해 값이 일치하는 정도를 보여주는 지표이다.
    범주간 차이의 비중을 제곱하므로 q(quadratic)가 붙은 것.
    '''
    O = confusion_matrix(act,pred)
    O = np.divide(O,np.sum(O))
    
    W = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            W[i][j] = ((i-j)**2)/((n-1)**2)
            
    act_hist = np.histogram(act,bins=n,range=hist_range)[0]
    prd_hist = np.histogram(pred,bins=n,range=hist_range)[0]
    
    E = np.outer(act_hist,prd_hist)
    E = np.divide(E,np.sum(E))
    
    num = np.sum(np.multiply(W,O))
    den = np.sum(np.multiply(W,E))
        
    return 1-np.divide(num,den)
    

필요한 csv 파일들을 읽어들인다.

In [ ]:
train = pd.read_csv('/kaggle/input/data-science-bowl-2019/train.csv')
train_labels = pd.read_csv('/kaggle/input/data-science-bowl-2019/train_labels.csv')
specs = pd.read_csv('/kaggle/input/data-science-bowl-2019/specs.csv')
test = pd.read_csv('/kaggle/input/data-science-bowl-2019/test.csv')
submission = pd.read_csv('/kaggle/input/data-science-bowl-2019/sample_submission.csv')

In [ ]:
# encode title
# 'title' 항목을 activity라고 명명하고 0부터 시작하는 숫자 값으로 변환할 수 있는 dictionary를 만든다.
list_of_user_activities = list(set(train['title'].value_counts().index).union(set(test['title'].value_counts().index)))
activities_map = dict(zip(list_of_user_activities, np.arange(len(list_of_user_activities))))

# train/test 데이터의 title을 숫자값으로 치환한다.
train['title'] = train['title'].map(activities_map)
test['title'] = test['title'].map(activities_map)
train_labels['title'] = train_labels['title'].map(activities_map)

In [ ]:
train.head(30)

In [ ]:
train_labels.head()

In [ ]:
print("list_of_user_activities:")
print(list_of_user_activities)
print("activities_map:")
print(activities_map)

activity(원래 title)마다 해당하는 win_code를 만든다.<br>
'Bird Measurer (Assessment)'만 4110이고 나머지는 4100임

In [ ]:
win_code = dict(zip(activities_map.values(), (4100*np.ones(len(activities_map))).astype('int')))
win_code[activities_map['Bird Measurer (Assessment)']] = 4110
print(win_code)

In [ ]:
train['timestamp'] = pd.to_datetime(train['timestamp'])
test['timestamp'] = pd.to_datetime(test['timestamp'])

no intersection between installation ids of train and test

train데이터에서 accuracy_group 추출

make_session_data()는 각각의 installation_id에 해당하는 event를 분석해서 accuracy_group을 구하는 동작을 한다.<br>
installation_id로 groupby()를 행한 sub dataframe을 인자로 넘긴다.

In [ ]:
def make_session_data(user_sample, test_set=False):
    '''
    user_sample : train.groupby('installation_id', sort=False)
        installation_id로 묶인 뭉텅이    
    
    test_set 인 경우 마지막 assessment만 남긴다.
    '''
    last_activity = 0
    user_activities_count = {'Clip':0, 'Activity': 0, 'Assessment': 0, 'Game':0}
    accuracy_groups = {0:0, 1:0, 2:0, 3:0}
    all_assessments = [] # 모든 assessment 세션의 요약정보(accuracy_group을 포함한 여러 column이 있음)
    accumulated_accuracy_group = 0
    accumulated_accuracy=0
    accumulated_correct_attempts = 0 
    accumulated_uncorrect_attempts = 0 
    accumulated_actions = 0
    counter = 0
    durations = []
    
    for i, session in user_sample.groupby('game_session', sort=False): #game_session 기준으로 또 group 
        """
        session 기준으로 묶으면 한 session 에서 일어난 이벤트들이 나열된 dataframe을 얻을 수 있다.
        결국 각각의 session마다 처리를 하는 code block임.
        """
        # type : Clip/Activity/Assessment/Game
        # title : 여기서는 숫자로 변경되어 있음
        session_type = session['type'].iloc[0]
        session_title = session['title'].iloc[0]
        if test_set == True:
            second_condition = True
        else:
            if len(session)>1:
                second_condition = True
            else:
                second_condition= False
        
        # session_type이 Assessment인 경우만 필요하다.
        # session이 Assessment(평가) 타입일 경우 여기 성공/실패 등의 정보가 있다.
        # 'Assessment' 세션의 이벤트들을 분석해서 데이터를 생성한다.
        if (session_type == 'Assessment') & (second_condition):
            '''
            'Assessment' 세션에서 일어난 이벤트들의 목록
            '''
            all_attempts = session.query(f'event_code == {win_code[session_title]}') #event_code가 win_code(하나빼고 4100)인 것을 모은다.
            true_attempts = all_attempts['event_data'].str.contains('true').sum() # 성공횟수(event_data에 'true'문자열이 잇는가?)
            false_attempts = all_attempts['event_data'].str.contains('false').sum() # 실패횟수(event_data에 'false'문자열이 잇는가?)
            #print("", all_attempts.shape[0], "true:", true_attempts, "false:", false_attempts)
            features = user_activities_count.copy()
            features['session_title'] = session_title
            features['accumulated_correct_attempts'] = accumulated_correct_attempts
            features['accumulated_uncorrect_attempts'] = accumulated_uncorrect_attempts
            accumulated_correct_attempts += true_attempts 
            accumulated_uncorrect_attempts += false_attempts
            if durations == []:
                features['duration_mean'] = 0
            else:
                features['duration_mean'] = np.mean(durations)            
            durations.append((session.iloc[-1, 2] - session.iloc[0, 2] ).seconds) #마지막의 timestamp(2, 세번째 column)에서 처음을 빼서 session의 실행시간을 얻는다.
            features['accumulated_accuracy'] = accumulated_accuracy/counter if counter > 0 else 0
            # 정확도를 계산한다.
            #print("true_attempts:", true_attempts, "false_attempts:", false_attempts)
            accuracy = true_attempts/(true_attempts+false_attempts) if (true_attempts+false_attempts) != 0 else 0
            accumulated_accuracy += accuracy
            '''
            accuracy_group:
            session 기준으로 accuracy_group를 계산할 수 있다.
                3: the assessment was solved on the first attempt
                2: the assessment was solved on the second attempt
                1: the assessment was solved after 3 or more attempts
                0: the assessment was never solved
                
            코드를 보면 평균내서 하는 것 같은데, 아마 1개의 true와 0개 이상의 false 혹은
            0개의 true와 여러개의 false가 있는 식인듯.
            '''
            if accuracy == 0: # 해결 못했다.
                features['accuracy_group'] = 0
            elif accuracy == 1: #실패가 없음
                features['accuracy_group'] = 3
            elif accuracy == 0.5: # 한번 실패하고 성공
                features['accuracy_group'] = 2
            else: # 3번 이상 실패함.
                features['accuracy_group'] = 1
                
            # update() : modify in place using non-NA values from another dataframe
            # 같은 column이 있으면 넘어온 dataframe값으로 대체한다.
            features.update(accuracy_groups)
            features['accumulated_accuracy_group'] = accumulated_accuracy_group/counter if counter > 0 else 0
            features['accumulated_actions'] = accumulated_actions
            accumulated_accuracy_group += features['accuracy_group']
            accuracy_groups[features['accuracy_group']] += 1
            if test_set == True:
                all_assessments.append(features)
            else:
                if true_attempts+false_attempts > 0:
                    all_assessments.append(features)
                
            counter += 1

        accumulated_actions += len(session)
        if last_activity != session_type:
            user_activities_count[session_type] += 1
            last_activitiy = session_type
    
    """
    FIXME: 여기는 check가 필요함.    
    if it't the test_set, only the last assessment must be predicted, the previous are scraped
    
    test 데이터의 경우 기기(installation_id)의 마지막 assessment의 accuracy_group을 예측해야 한다.
    그러므로 마지막 assessment항목만 남기는 코드가 있는건데... 
    그러면 그 앞의 데이터들은 아무런 필요가 없는 것인지...?
    다른 평가가 이루어진(Assessment이고 event_data에 true/false 있음) 세션도 많아서...
    
    일단 다른 커널들을 보면 아래처럼 처리(제거함)하는 것 같다.
    점수가 높은 노트북들에서도 모두 날리므로 날리는것이 맞는듯.
    날리는 노트북들:    
        https://www.kaggle.com/artgor/quick-and-dirty-regression
        https://www.kaggle.com/hengzheng/bayesian-optimization-seed-blending
        
    """
    if test_set:
        return all_assessments[-1]
    
    # in the train_set, all assessments goes to the dataset
    return all_assessments

make_session_data()함수로 train 데이터를 session에 대한 요약 정보(accuracy_group 항목 생성해서 포함) 데이터로 변환한다.

In [ ]:
def make_assessment_group_data(df, is_test_set=False):
    compiled_data = list() 
    total_cnt = df.installation_id.unique().shape[0]
    for i, (ins_id, user_sample) in tqdm(enumerate(df.groupby('installation_id', sort=False)), total=total_cnt):        
        sdata = make_session_data(user_sample, test_set=is_test_set)
        
        if type(sdata) is list:
            compiled_data += sdata
        else:
            compiled_data.append(sdata)

    newTrain = pd.DataFrame(compiled_data)    
    return newTrain

In [ ]:
new_train = make_assessment_group_data(train)

Below are the features I have generated. Note that all of them are **prior** to each event. For example, the first row shows **before** this assessment, the player have watched 3 clips, did 3 activities, played 4 games and solved 0 assessments, so on so forth.

**new_train columns:**

- 'Clip'
- 'Activity'
- 'Assessment'
- 'Game'
- 'session_title'
- 'accumulated_correct_attempts'
- 'accumulated_uncorrect_attempts'
- 'duration_mean'
- 'accumulated_accuracy'
- 'accuracy_group'
- 0
- 1
- 2
- 3
- 'accumulated_accuracy_group'
- 'accumulated_actions'

In [ ]:
new_train.head()

## Model

In [ ]:
all_features = [x for x in new_train.columns if x not in ['accuracy_group']]
print(all_features)

- X: 'accuracy_group'를 제외한 모든것.
- y: 'accuracy_group'

In [ ]:
new_train.head()

In [ ]:
all_features = [x for x in new_train.columns if x not in ['accuracy_group']]
cat_features = ['session_title'] #train의 'title' 항목
X, y = new_train[all_features], new_train['accuracy_group']
del train

In [ ]:
X.head()

In [ ]:
X.columns.tolist()

In [ ]:
y.head()

In [ ]:
def make_classifier():
    clf = CatBoostClassifier(
        loss_function='MultiClass',    
        task_type="CPU",
        learning_rate=0.01,
        iterations=2000,
        od_type="Iter",
        early_stopping_rounds=500,
        random_seed=2019)
    
    return clf

In [ ]:
# CV
from sklearn.model_selection import KFold
oof = np.zeros(len(X))
NFOLDS = 5
folds = KFold(n_splits=NFOLDS, shuffle=True, random_state=2019)

training_start_time = time()
for fold, (trn_idx, test_idx) in enumerate(folds.split(X, y)):
    start_time = time()
    print(f'Training on fold {fold+1}')
    clf = make_classifier()
    clf.fit(X.loc[trn_idx, all_features], y.loc[trn_idx], eval_set=(X.loc[test_idx, all_features], y.loc[test_idx]),
                          use_best_model=True, verbose=500, cat_features=cat_features)
    
#     preds += clf.predict(X_test).reshape(len(X_test))/NFOLDS
    oof[test_idx] = clf.predict(X.loc[test_idx, all_features]).reshape(len(test_idx))
    
    print('Fold {} finished in {}'.format(fold + 1, str(datetime.timedelta(seconds=time() - start_time))))
    
print('-' * 30)
print('OOF QWK:', qwk(y, oof))
print('-' * 30)

Note that Cross validation is only for the feature engineering part and you don't actually need it if you want to submit the results. You can safely comment it out. 

CV로 하는 것은 사용하지 않고, 전체 데이터로 한번만 train한 모델을 사용한다고 한다.
기존 이미지 판별 문제와는 다른 방법인 것 같은데... 
이상하네...?

In [ ]:
# train model on all data once
clf = make_classifier()
clf.fit(X, y, verbose=500, cat_features=cat_features)

del X, y

위 train 데이터를 생성하던 것과 동일한 방법으로 test데이터를 사용해서 생성한다.

## Submission

test 데이터로 input을 생성한다.<br>
train과 동일한 방법을 사용하되, make_session_data()호출시 test_set=True 로 한다.

In [ ]:
X_test = make_assessment_group_data(test, is_test_set=True)
X_test.head()

In [ ]:
X_test.head()

In [ ]:
test_columns = X_test.columns.tolist()
print(len(test_columns), "columns:\n", test_columns)

In [ ]:
X_test = X_test[all_features]

In [ ]:
# make predictions on test set once
preds = clf.predict(X_test)

In [ ]:
X_test.head(10)

pred를 accuracy_group으로 정해서 csv파일을 생성한다.<br>
submission은 sample_submission.csv파일을 읽어들인 것인데, sample_submission의 installation_id 순서를 그대로 사용한다.<br>
위에서 groupby(sort=False)여서 문제 없는듯.

In [ ]:
submission['accuracy_group'] = np.round(preds).astype('int') #이 부분에 OptimizedRounder가 필요함.
submission.to_csv('submission.csv', index=None)
submission.head()

In [ ]:
submission['accuracy_group'].plot(kind='hist')

In [ ]:
train_labels['accuracy_group'].plot(kind='hist')

In [ ]:
pd.Series(oof).plot(kind='hist')